## Hands-On Workshop
## Large-Scale Geospatial Analytics With Wherobots, Neo4j, & The PyData Ecosystem

This workshop demonstrates how to use 

This notebook covers :

* Calculating bird species range using Spatial SQL
* Building and analyzing a bird species interaction graph with Neo4j
* Analyzing raster data (precipitation) to enrich our graph 
* Searching Overture Maps point of interest data to find National Parks in each species range

## Import Dependencies



In [ ]:
from sedona.spark import *
import geopandas
import json

## Configure SedonaContext

In [ ]:
# Configure SedonaContext, specify credentials for AWS S3 bucket(s) (optional)

config = SedonaContext.builder(). \
    config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider"). \
    getOrCreate()

sedona = SedonaContext.create(config)

## Calculating Bird Species Range

We'll load a dataset of bird species observations, then calculate the range of each species using Spatial SQL.

Our data comes from [Bird Buddy](https://live.mybirdbuddy.com/) which makes a smart bird feeder than can identify bird species and (optionally) report their location.

![](https://wherobots.com/wp-content/uploads/2023/11/bird_buddy1.png)


The data is availabe in CSV format which we can load using Sedona's built-in CSV reader.

In [ ]:
BB_S3_URL = "s3://wherobots-examples/data/examples/birdbuddy_oct23.csv"
bb_df = sedona.read.format('csv').option('header','true').option('delimiter', ',').load(BB_S3_URL)
bb_df.show(5, truncate=False)

Note, however that we'll need to explicitly convert the `anonymized_latitude` and `anonymized_longitude` fields into a `geometry` field to work with the point locations. To do this we'll use the [`ST_Point` Spatial SQL function.](https://docs.wherobots.services/1.2.2/references/sedonadb/vector-data/Constructor/#st_point)

In [ ]:
bb_df = bb_df.selectExpr('ST_Point(CAST(anonymized_longitude AS float), CAST(anonymized_latitude AS float)) AS location', 'CAST(timestamp AS timestamp) AS timestamp', 'common_name', 'scientific_name')
bb_df.createOrReplaceTempView('bb')
bb_df.show(15, truncate=False)

In [ ]:
bb_df.printSchema()


In [ ]:
bb_df.count()


We can visualize our species observation data using [`SedonaKepler`, the Kepler GL integration for Sedona.](https://docs.wherobots.services/1.2.2/references/sedonadb/vector-data/Visualization_SedonaKepler/?h=sedonakepler)

In [ ]:
SedonaKepler.create_map(bb_df.sample(0.005), name="Bird species observations")

## Calculating Species Range Extent

Our goal is to build a graph of species interactions. To do this we first must calculate the range extent of each species based on the point observation data. To do this we will calculate a convex hull for each species' observations. A convex hull is the smallest polygon that can be drawn around a group of points that will enclose each point.

![](https://wherobots.com/wp-content/uploads/2024/03/HRe9kN3VEohA2YVqRmpW2Eb6YKOuhvd4qA.png)

[Image Credit - EcoCommons](https://support.ecocommons.org.au/support/solutions/articles/6000254290-convex-hull)

To do this we will make use of the [ST_ConvexHull spatial SQL function](https://docs.wherobots.services/1.2.2/references/sedonadb/vector-data/Function/?h=st_convexhull#st_convexhull), along with an aggregation and `GROUP BY` statement.

In [ ]:
# To simply our initial analysis we'll filter for a subset of species.

range_df = sedona.sql("""
    SELECT common_name, COUNT(*) AS num, ST_ConvexHull(ST_Union_aggr(location)) AS geometry 
    FROM bb 
    WHERE common_name IN ('california towhee', 'steller’s jay', 'mountain chickadee', 'eastern bluebird', 'wood thrush', 'yellow headed blackbird', 'spot breasted oriole', 
      'red cockaded woodpecker', 'northern red bishop', 'red naped sapsucker', 'western meadowlark', 'lazuli bunting', 'clark’s nutcracker', 'gray crowned rosy finch', 'california quail',
      'boreal chickadee', 'acorn woodpecker', 'townsend’s warbler', 'gambel’s quail', 'scott’s oriole', 'cassin’s finch', 'brown headed nuthatch', 'pygmy nuthatch', 'pinyon jay', 'florida scrub jay') 
    GROUP BY common_name 
    ORDER BY num DESC
""")
range_df.show(30)

In [ ]:
range_df.createOrReplaceTempView("ranges")

In [ ]:
range_df = range_df.cache()

In [ ]:
SedonaKepler.create_map(df=range_df, name="Bird species range")


## Determine Species Range Intersection

The next step in building our species interaction graph is to determine which species range's intersect. To do this we will use the [`ST_Intersects` predicate](https://docs.wherobots.services/1.2.2/references/sedonadb/vector-data/Predicate/#st_intersects) Spatial SQL function. We'll also calculate the centroid of each range using the [`ST_Centroid` function](https://docs.wherobots.services/1.2.2/references/sedonadb/vector-data/Function/#st_centroid) which will allow us to represent the species range as a single point geometry, instead of a polygon.

In [ ]:
intersect_df = sedona.sql("""
    WITH birds AS (SELECT * FROM ranges)
    SELECT
      birds.common_name, 
      ST_Centroid(any_value(birds.geometry)) AS centroid, 
      collect_list(ranges.common_name) AS intersects
    FROM ranges, birds
    WHERE 
      ST_Intersects(birds.geometry, ranges.geometry) 
      AND NOT birds.common_name=ranges.common_name
    GROUP BY birds.common_name
""")

In [ ]:
intersect_df.show()

In [ ]:
intersect_df.createOrReplaceTempView("intersects")

## Build A Graph Of Bird Species Interactions

Next, we'll load into Neo4j our bird species data, creating a graph of bird species that have overlapping range. This will allow us to answer questions related to ecology, disease transmission, and conservation.

### Define The Graph Model

The first step when building a graph is always to define the graph data model. A great tool to sketch out our graph data model is [Arrows.app](https://arrows.app). In this case our data model will be fairly simple:

![](https://wherobots.com/wp-content/uploads/2024/03/bird_species_range.png)

### Working With The Neo4j Python Driver

We can use the [official Neo4j Python driver](https://neo4j.com/docs/python-manual/current/) to execute Cypher queries in a Neo4j database from a Python environment, like this hosted Jupyter notebook.


In [ ]:
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase

In [ ]:
# Fill in your Neo4j Aura credentials
URI = "neo4j+s://<YOUR_AURA_INSTANCE_HERE>.databases.neo4j.io"
AUTH = ("neo4j", "<YOUR_PASSWORD_HERE>")

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    records, summary, keys = driver.execute_query(
        "MATCH (n) RETURN COUNT(n) AS num"
    )
    for record in records:
        print(record)

### Batch Neo4j Imports Using Geopandas and GeoJSON

When loading large amounts of data into Neo4j we typically want to batch operations across transactions to efficiently handle memory usage.  By iterating over a DataFrame object and operating in batches we can ensure we don't build up too much transaction state in memory. Note that with the size of our bird ranges in this example this isn't really necessary, but this approach will be useful when working with larger data sizes. 

We'll make use of [GeoPandas](https://geopandas.org/en/stable/index.html), which extends the Pandas DataFrame data structure to add geospatial functionality and integration with Python tooling. 

In [ ]:
birds_gdf = geopandas.GeoDataFrame(intersect_df.toPandas(), geometry="centroid")

In [ ]:
birds_gdf

GeoPandas can convert rows to GeoJSON, which we'll pass as a parameter with our Cypher query to create the nodes and relationships in our graph.

In [ ]:
json.loads(birds_gdf[0:1].to_json())['features']

Next, we define a parameterized Cypher query to create `Species` nodes for each bird range and then iterate over the `intersects` array, creating the `RANGE_OVERLAP` relationship for each intersecting species.

In [ ]:
neo4j_query = """
UNWIND $rows AS row
MERGE (s:Species {common_name: row.properties.common_name})
SET s.centroid = Point({longitude: row.geometry.coordinates[0], latitude: row.geometry.coordinates[1]})
WITH s, row
UNWIND row.properties.intersects AS bird
MERGE (b:Species {common_name: bird})
MERGE (s)-[:RANGE_OVERLAP]-(b)
RETURN COUNT(*) AS total
"""

In [ ]:
def insert_data(tx, query, rows, batch_size=1000):
    total = 0
    batch = 0
    while batch * batch_size < len(rows):
        print(batch)
        print(batch_size)
        results = tx.run(query, parameters = {
            'rows': json.loads(rows[batch * batch_size : (batch + 1) * batch_size].to_json())['features']
        }).data()
        print(results)
        total += results[0]['total']
        batch += 1
    

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        session.execute_write(insert_data, neo4j_query, birds_gdf)

## Exercise - Graph Analysis With Cypher & Neo4j



## Working With Raster Data

So far we've been working with *vector* geospatial data - geometries and their associated attributes. Another important type of geospatial data is *raster* data.


![](https://wherobots.com/wp-content/uploads/2024/02/raster_concept.png)


### Calculating Precipitation 

Understanding the climate of each species' range can be an important aspect of ecological analysis. Using historical climate data from [WorldClim](https://www.worldclim.org/data/worldclim21.html) we can calculate statistics about each area.

#### Zonal Statistics

Zonal statistics are calculations applied to raster data, using the bounds of another raster or vector geometry. For example, calculating the average value of all cells within the bounds of a city.

In Sedona we can calculate zonal statistics using the [`RS_ZonalStats` Spatial SQL function.](https://docs.wherobots.services/1.2.2/references/sedonadb/raster-data/Raster-operators/#rs_zonalstats)

![](https://wherobots.com/wp-content/uploads/2024/03/Zonal-statistics-operation.png)

In [ ]:
PREC_URL = "s3://wherobots-examples/data/examples/world_clim/wc2.1_10m_prec" #/wc2.1_10m_prec_01.tif
rawDf = sedona.read.format("binaryFile").load(PREC_URL + "/*.tif")
rawDf.createOrReplaceTempView("rawdf")

In [ ]:
rasterDf = sedona.sql("""
SELECT 
  RS_FromGeoTiff(content) AS raster, 
  Int(regexp_extract(path, '(.*)([0-9]{2}).tif', 2)) AS month
FROM rawdf
""")

rasterDf.createOrReplaceTempView("prec")
rasterDf.printSchema()

In [ ]:
world_prec_df = sedona.sql("""
SELECT 
  sum(RS_ZonalStats(prec.raster, ranges.geometry, 1, 'avg', true)) AS yearly_avg_prec,  
  any_value(ranges.geometry) AS geometry, 
  ranges.common_name
FROM prec, ranges
GROUP BY common_name
ORDER BY yearly_avg_prec DESC
""")
world_prec_df.dropna().show()

In [ ]:
SedonaKepler.create_map(world_prec_df, name="Precipitation")

### Enriching The Species Interaction Graph

Let's update the species interaction graph with this precipiation information, adding a property `annual_precip`.

![](https://wherobots.com/wp-content/uploads/2024/03/Untitled-graph.png)

In [ ]:
precip_gdf = geopandas.GeoDataFrame(world_prec_df.toPandas(), geometry="geometry")

In [ ]:
precip_update_query = """
UNWIND $rows AS row
MATCH (s:Species {common_name: row.properties.common_name})
SET s.annual_precip = row.properties.yearly_avg_prec
RETURN COUNT(*) AS total
"""

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        session.execute_write(insert_data, precip_update_query, precip_gdf)

## Overture Example

Understanding what protected lands may be within the range of each species is important for conservation planning. We can use [Overture Maps data](https://docs.overturemaps.org/) available with the [Wherobots Open Data Catalog](https://docs.wherobots.services/1.2.2/tutorials/opendata/introduction/) to search for national parks within each species range.

In [ ]:
# find national parks within each species range

In [ ]:
sedona.table('wherobots_open_data.overture_2024_02_15.base_landUse').printSchema()

In [ ]:
park_df = sedona.sql("""
SELECT 
  names.primary AS park, 
  collect_list(common_name) AS species, 
  any_value(wherobots_open_data.overture_2024_02_15.base_landUse.geometry) AS geometry
FROM wherobots_open_data.overture_2024_02_15.base_landUse, ranges
WHERE ST_Intersects(ranges.geometry, wherobots_open_data.overture_2024_02_15.base_landUse.geometry) AND names.primary IS NOT NULL
AND wherobots_open_data.overture_2024_02_15.base_landUse.class = "nationalPark"
GROUP BY park
""")

In [ ]:
park_df.show()

In [ ]:
park_df.count()

In [ ]:
park_map = SedonaKepler.create_map()
SedonaKepler.add_df(park_map, park_df, name="Parks")
SedonaKepler.add_df(park_map, range_df, name="Birds")
park_map

In [ ]:
park_df = sedona.sql("""
SELECT 
  names.primary AS park, 
  collect_list(common_name) AS species, 
  ST_Centroid(any_value(wherobots_open_data.overture_2024_02_15.base_landUse.geometry)) AS geometry
FROM wherobots_open_data.overture_2024_02_15.base_landUse, ranges
WHERE ST_Intersects(ranges.geometry, wherobots_open_data.overture_2024_02_15.base_landUse.geometry) AND names.primary IS NOT NULL
AND wherobots_open_data.overture_2024_02_15.base_landUse.class = "nationalPark"
GROUP BY park
""")

### Adding National Parks To The Graph

![](https://wherobots.com/wp-content/uploads/2024/03/Bird_Species_Graph.png)

In [ ]:
park_gdf = geopandas.GeoDataFrame(park_df.toPandas(), geometry="geometry")

In [ ]:
park_update_query = """
UNWIND $rows AS row
MERGE (n:NationalPark {name: row.properties.park})
SET n.centroid = Point({longitude: row.geometry.coordinates[0], latitude: row.geometry.coordinates[1]})
WITH n, row
UNWIND row.properties.species AS bird
MATCH (s:Species {common_name: bird})
MERGE (s)<-[:WITHIN_RANGE]-(n)
RETURN COUNT(*) AS total
"""

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    with driver.session() as session:
        session.execute_write(insert_data, park_update_query, park_gdf)